<a href="https://colab.research.google.com/github/bevislau/Data_Project/blob/main/Predicting_b2c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting B2C Visitor Purchases – Classification Modeling with BigQuery ML

This project applies a **binary classification model** using logistic regression to predict whether a B2B website visitor will convert into a customer. By analyzing user behavior data, the goal is to identify key factors that influence purchase intent and to build a predictive model that can support marketing and sales strategies.

In this case study, we use the **BigQuery ML public sample dataset** provided by Google:  
🔗 [BigQuery ML Templates Dataset](https://github.com/GoogleCloudPlatform/bigquery-ml-templates/tree/master/dataset)

The dataset captures behavioral signals from B2B visitors, such as session activity, engagement time, and visit source, which are leveraged to train the model and make predictions.

---

## 🔑 Key Steps in the Project

1. **Data Exploration & Preparation**  
   Understand the schema, clean the data, and define the target variable.

2. **Feature Engineering**  
   Derive meaningful behavioral features from raw logs to feed into the model.

3. **Model Creation**  
   Use logistic regression via BigQuery ML to build a classification model that predicts conversions.

4. **Model Evaluation**  
   Assess model performance using evaluation metrics like ROC AUC and precision-recall.

5. **Model Prediction**  
   Apply the trained model to predict user conversions for a selected day, informing future marketing activities.

6. **Recommendations & Next Steps**  
   Suggest ways to apply the model in real-world campaigns and propose future enhancements.


In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project='gifted-country-461816-i9')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Then set your GCP project explicitly:
PROJECT_ID = "gifted-country-461816-i9"

# If you’re using %bigquery magic, set it like this:
%env GOOGLE_CLOUD_PROJECT=gifted-country-461816-i9

# Data Exploration & Preparation

## Events data in GA4



In [ ]:
query = """
SELECT event_name , COUNT(*)
FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
GROUP BY event_name
ORDER BY COUNT(*) DESC

"""

df = client.query(query).to_dataframe()
display(df)

##Pearson Correlation with events

In [ ]:
query = """
SELECT
  user_pseudo_id,
  event_name,
  COUNT(*) AS event_count
FROM
  `gifted-country-461816-i9.ga4_sample_data.events_*`
GROUP BY
  user_pseudo_id, event_name
ORDER BY COUNT(*) DESC
"""

df_raw = client.query(query).to_dataframe()

# Step 2: Pivot events into wide format (columns = event types)
df_pivot = df_raw.pivot_table(index='user_pseudo_id',
                               columns='event_name',
                               values='event_count',
                               fill_value=0)

# Step 3: Compute correlation matrix
corr_matrix = df_pivot.corr(method='pearson')

# Step 4: Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title("Pearson Correlation Between Event Types")
plt.tight_layout()
plt.show()

## User Demographics

In [ ]:
query = """
SELECT geo.country,
COUNT(DISTINCT CONCAT(user_pseudo_id,event_bundle_sequence_id)) AS sessions_count ,
COUNT(DISTINCT user_pseudo_id) AS user_count ,
COUNT(DISTINCT CONCAT(user_pseudo_id,event_bundle_sequence_id))/COUNT(DISTINCT user_pseudo_id) AS Sessions_per_Users

FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
GROUP BY geo.country
ORDER BY COUNT(DISTINCT CONCAT(user_pseudo_id,event_bundle_sequence_id)) DESC

"""

df = client.query(query).to_dataframe()
display(df)

# Feature Engineering

To make it simply, we will predict whether the users are covnerted from first seesions and then use it to as an indication

Approach:

-Determine the first sessions users engaged with the website by MIN(event_timestamp) or user_first_touch_stamp

-Build the customer behaviour with events under multiple sessions or only 1

-Combine Behavioural and Demographics data with user_id for prediction

In [ ]:
%%bigquery df --project gifted-country-461816-i9
---Find out the first sessions user visit
WITH first_time_user AS(
SELECT
  user_pseudo_id,
  MIN(event_timestamp) AS first_time,
  CONCAT(user_pseudo_id,MIN(event_timestamp)),
  COUNT(*)
FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
WHERE event_name = 'page_view'
GROUP BY user_pseudo_id
)
--- Return 1 when user have the purchase in the same sessions
--- user_pseudo_id , label
--- WHERE: first sessions


SELECT
  CONCATE(user_pseudo_id,first_time),
  CASE WHEN event_name = 'Purchase' THEN 1 ELSE 0 END
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` a
JOIN first_time_user ft ON
CONCAT(a.user_pseudo_id,MIN(a.event_timestamp)) = CONCAT(ft.user_pseudo_id,ft.first_time)




In [ ]:
display(df)

In [ ]:
%%bigquery df --project gifted-country-461816-i9
SELECT
  user_pseudo_id,
  TIMESTAMP_MICROS(MIN(event_timestamp)) AS first_time,
  TIMESTAMP_MICROS(user_first_touch_timestamp)AS user_first_touch_timestamp
FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
WHERE event_name = 'page_view'
GROUP BY user_pseudo_id,TIMESTAMP_MICROS(user_first_touch_timestamp)
LIMIT 10

In [ ]:
display(df)

From the result, we can confirm the event_timestamp would equal to user_first_touch_timestamp if this is the first time customer enter the site

In GA4, the typical approach will be to combine User pseudo id with Sessions id to find the earliest and we would consider consent status as well


In [ ]:
%%bigquery df --project gifted-country-461816-i9
---first sessions start time
SELECT
    user_pseudo_id,
    (SELECT value.int_value
     FROM UNNEST(event_params)
     WHERE key = 'ga_session_id' ) AS session_id,
    MIN(event_timestamp) AS session_start_time,
    user_first_touch_timestamp
  FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE event_name = 'page_view'
  GROUP BY user_pseudo_id, session_id,user_first_touch_timestamp
  ORDER BY session_start_time ASC

In [ ]:
display(df)

In [ ]:
%%bigquery df --project gifted-country-461816-i9
---Step 1: Find out the first sessions user visit period
---Step 2: ues the WHERE clause, only include those first sessions, and count the event

WITH first_time_sessions AS(
SELECT
  CONCAT(user_pseudo_id,
  (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  ) AS first_user_id_and_session
FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
WHERE event_name = 'page_view' AND event_timestamp = user_first_touch_timestamp
)

SELECT
m.user_pseudo_id,
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN first_time_sessions f
ON CONCAT(m.user_pseudo_id,
  (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  )  = f.first_user_id_and_session
GROUP BY m.user_pseudo_id
ORDER BY page_view_count DESC,view_item_count DESC,add_to_cart_count DESC,scroll_count DESC



In [ ]:
display(df)


### Double check the work from the above for the highest activity user

In [ ]:
%%bigquery df --project gifted-country-461816-i9
---Double check the work from the above for the highest activity user

WITH first_time_sessions AS(
SELECT
  CONCAT(user_pseudo_id,
  (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  ) AS first_user_id_and_session
FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
WHERE event_name = 'page_view' AND event_timestamp = user_first_touch_timestamp
)

SELECT
m.user_pseudo_id,
event_name,
(SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN first_time_sessions f
ON CONCAT(m.user_pseudo_id,
  (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  )  = f.first_user_id_and_session
WHERE m.user_pseudo_id = '23345428.2948094181'

In [ ]:
display(df)

## To consider more than ther first sessions behaviour, we start ranking the sessions and gather the user behaviour

In [ ]:
%%bigquery df --project gifted-country-461816-i9
-- with first_session start time
WITH session_start_times AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS session_id,
    MIN(event_timestamp) AS session_start_time
  FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE event_name = 'page_view'
  GROUP BY user_pseudo_id, session_id

),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
)

-- Get only second session or LESS
SELECT *
FROM ranked_sessions
WHERE session_rank <= 2
ORDER BY user_pseudo_id ,session_rank ASC



In [ ]:
display(df)

In [ ]:
%%bigquery df --project gifted-country-461816-i9
-- with first_session start time
WITH session_start_times AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS session_id,
    MIN(event_timestamp) AS session_start_time
  FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE event_name = 'page_view'
  GROUP BY user_pseudo_id, session_id

),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
),

user_firsttwo_sessions AS(
-- Get only second session or less
SELECT *
FROM ranked_sessions
WHERE session_rank <= 2
ORDER BY user_pseudo_id ,session_rank ASC
)

SELECT
m.user_pseudo_id,
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
SUM(CAST((SELECT value.int_value
            FROM UNNEST(event_params)
            WHERE key = 'engagement_time_msec') AS INT64)) AS total_engagement_time_msec,
CASE WHEN (COUNTIF (event_name = 'purchase') >= 1 ) THEN 1 ELSE 0 END AS will_buy_on_firsttwo_Sessions
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN user_firsttwo_sessions f
ON (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  = f.session_id
GROUP BY m.user_pseudo_id
ORDER BY will_buy_on_firsttwo_Sessions DESC , page_view_count DESC,view_item_count DESC,add_to_cart_count DESC,scroll_count DESC



In [ ]:
display(df)

## Comparsion with purchaser and non-purchasesr under 2 sessions

In [ ]:
%%bigquery df --project gifted-country-461816-i9
-- with first_session start time
WITH session_start_times AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS session_id,
    MIN(event_timestamp) AS session_start_time
  FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE event_name = 'page_view'
  GROUP BY user_pseudo_id, session_id

),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
),

user_firsttwo_sessions AS(
-- Get only second session or less
SELECT *
FROM ranked_sessions
WHERE session_rank <= 2
ORDER BY user_pseudo_id ,session_rank ASC
),

user_profiling AS(
SELECT
m.user_pseudo_id,
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
CASE WHEN (COUNTIF (event_name = 'purchase') >= 1 ) THEN 1 ELSE 0 END AS will_buy_on_firsttwo_Sessions
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN user_firsttwo_sessions f
ON (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  = f.session_id
GROUP BY m.user_pseudo_id
ORDER BY will_buy_on_firsttwo_Sessions DESC , page_view_count DESC,view_item_count DESC,add_to_cart_count DESC,scroll_count DESC
)

SELECT will_buy_on_firsttwo_Sessions, COUNT(*)
FROM user_profiling
GROUP BY will_buy_on_firsttwo_Sessions


In [ ]:
display(df)

### From the result, the % of purchaser is 3247/(266559+3247) = 1.2% purchasers.

### As the % purchasesr is quiet low, i have go ahead and widen the criteria with the sessions to 4

## Comparsion with purchaser and non-purchasesr under 4 sessions

In [ ]:
%%bigquery df --project gifted-country-461816-i9
-- with first_session start time
WITH session_start_times AS (
  SELECT
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS session_id,
    MIN(event_timestamp) AS session_start_time
  FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE event_name = 'page_view'
  GROUP BY user_pseudo_id, session_id

),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
),

user_firsttwo_sessions AS(
-- Get only second session or less
SELECT *
FROM ranked_sessions
WHERE session_rank <= 4
ORDER BY user_pseudo_id ,session_rank ASC
),

user_profiling AS(
SELECT
m.user_pseudo_id,
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'user_engagement') AS user_engagement_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
CASE WHEN (COUNTIF (event_name = 'purchase') >= 1 ) THEN 1 ELSE 0 END AS will_buy_on_firstfour_Sessions
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN user_firsttwo_sessions f
ON (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  = f.session_id
GROUP BY m.user_pseudo_id
)

SELECT will_buy_on_firstfour_Sessions, COUNT(*)
FROM user_profiling
GROUP BY will_buy_on_firstfour_Sessions


In [ ]:
display(df)

## From the result, the % of purchaser is higher 4069/ (265737+4069) = 1.51% purchasers.

# Model Creation

From the initial feature engineering, there are less behavioural data planned and later the model has been further fine-tuned


### 🎯 Label:
- `label`: 1 if the user triggered a **purchase event** in their first 4 sessions, 0 otherwise.

### 🧠 Features used for prediction:

#### 🧍‍♂️ User Behavior:
- `page_view_count`
- `view_item_count`
- `scroll_count`
- `user_engagement_count`
- `add_to_cart_count`
- `view_promotion_count`
- `total_engagement_time_msec` (New)

#### 🌍 Demographics & Technology:
- `device_category` (e.g., desktop, mobile) (New)
- `country` (geo-location during first session) (New)
- Unforunately we cannot include channel and source and medium as it is hiddne in the sample data

👉 Users with no `view_item`, `page_view`` activity were excluded to focus on **meaningfully engaged users**.

In [ ]:
%%bigquery df --project gifted-country-461816-i9

CREATE OR REPLACE TABLE `gifted-country-461816-i9.ga4_sample_data.ml_prepareddata` AS (

WITH events_flat AS (
  SELECT
    user_pseudo_id,
    event_timestamp,
    (SELECT value.int_value
     FROM UNNEST(event_params)
     WHERE key = 'ga_session_id' LIMIT 1) AS session_id
  FROM
    `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE
    event_name = 'page_view'
),

-- One row per session per user, with session start time
session_start_times AS (
  SELECT
    user_pseudo_id,
    session_id,
    MIN(event_timestamp) AS session_start_time
  FROM events_flat
  WHERE session_id IS NOT NULL
  GROUP BY user_pseudo_id, session_id
),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
),

user_firsttwo_sessions AS(
-- Get only second session or less
SELECT *
FROM ranked_sessions
WHERE session_rank <=4
ORDER BY user_pseudo_id ,session_rank ASC
),
user_metadata AS (
  SELECT
    user_pseudo_id,
    device.category AS device_category,
    geo.country AS country
  FROM (
    SELECT *,
      RANK() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC) AS session_rank
    FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
    WHERE event_name = 'session_start'
  )
  WHERE session_rank = 1
)

SELECT
m.user_pseudo_id,
--Behavioural data
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
COUNTIF(event_name = 'user_engagement') AS user_engagement_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'view_promotion') AS view_promotion_count,
SUM(CAST((SELECT value.int_value
            FROM UNNEST(event_params)
            WHERE key = 'engagement_time_msec') AS INT64)) AS total_engagement_time_msec,
--demographic
  um.device_category AS device_category,
  um.country AS country,

--label
CASE WHEN (COUNTIF (event_name = 'purchase') >= 1 ) THEN 1 ELSE 0 END AS label
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN user_firsttwo_sessions f
ON (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  = f.session_id
LEFT JOIN user_metadata um
ON  m.user_pseudo_id = um.user_pseudo_id
GROUP BY m.user_pseudo_id,um.device_category, um.country
HAVING view_item_count > 0 AND page_view_count > 0
)



In [ ]:
%%bigquery df --project gifted-country-461816-i9

CREATE OR REPLACE MODEL `gifted-country-461816-i9.ga4_sample_data.model`
OPTIONS(
  model_type='logistic_reg',
  auto_class_weights=TRUE,
  input_label_cols=['label']
)
AS
SELECT
  page_view_count,
  view_item_count,
  scroll_count,
  add_to_cart_count,
  total_engagement_time_msec,
  user_engagement_count,
  view_promotion_count,
  device_category,
  country,
  label
FROM `gifted-country-461816-i9.ga4_sample_data.ml_prepareddata`

# Model Evaluation

In [ ]:
%%bigquery df --project gifted-country-461816-i9

SELECT *
FROM ML.EVALUATE(
  MODEL `gifted-country-461816-i9.ga4_sample_data.model`)

In [ ]:
display(df)

## 📊 Model Evaluation Results

| Metric       | Value   |
|--------------|---------|
| Precision    | 0.5944  |
| Recall       | 0.6556  |
| Accuracy     | 0.8268  |
| F1 Score     | 0.6235  |
| Log Loss     | 0.4527  |
| ROC AUC      | 0.8737  |

---

### 🔍 Key Takeaways:
✅ **High Accuracy (83%)**  
> Most predictions are correct (Both Positive and Negatives). Strong generalisation

⚖️ **Balanced Precision & Recall**  
> Captures ~66% of purchasers (recall), and 59% of positive predictions are correct (precision).
> Recall shows we are able to find purchasers better, while maybe less as precision to get it right
> Work further to obtain a higher precision and recall

✅ **Strong ROC AUC (0.87)**  
> The model does a great job ranking likely purchasers — it is likely to rank a real buyer higher than non-buyer.
ideal for targeting and prioritisation.

🔁 **Improved F1 Score (~0.62)**  
> Indicates a fair balance between false positives and false negatives.


#Model Prediction

In [ ]:
query = """
CREATE OR REPLACE TABLE `gifted-country-461816-i9.ga4_sample_data.model_predictions` AS
SELECT
  predicted_label,
  label
FROM
  ML.PREDICT(
    MODEL `gifted-country-461816-i9.ga4_sample_data.model`,
    TABLE `gifted-country-461816-i9.ga4_sample_data.ml_prepareddata`
  )

"""

# Run the query
query_job = client.query(query)
query_job.result()  # Wait for job to finish
print("Table created successfully.")

In [ ]:
query = """
SELECT predicted_label, label
FROM `gifted-country-461816-i9.ga4_sample_data.model_predictions`
"""
df = client.query(query).to_dataframe()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Ensure data is in int format
y_true = df['label'].astype(int)
y_pred = df['predicted_label'].astype(int)

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display it with labels
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Purchase", "Purchase"])
disp.plot(cmap='Blues')

In [ ]:
%%bigquery df --project gifted-country-461816-i9


SELECT
  user_pseudo_id,
  page_view_count,
  view_item_count,
  scroll_count,
  user_engagement_count,
  view_promotion_count,
  total_engagement_time_msec,
  device_category,
  country,
  predicted_label,
  predicted_label_probs[SAFE_OFFSET(1)].prob AS purchase_probability

FROM ML.PREDICT(
  MODEL `gifted-country-461816-i9.ga4_sample_data.model`,
  (
    -- New customers feature data here, e.g. last day (20210131)
WITH events_flat AS (
  SELECT
    user_pseudo_id,
    event_timestamp,
    (SELECT value.int_value
     FROM UNNEST(event_params)
     WHERE key = 'ga_session_id' LIMIT 1) AS session_id
  FROM
    `gifted-country-461816-i9.ga4_sample_data.events_*`
  WHERE
    event_name = 'page_view'
    AND _table_suffix = '20210131'
),

-- One row per session per user, with session start time
session_start_times AS (
  SELECT
    user_pseudo_id,
    session_id,
    MIN(event_timestamp) AS session_start_time
  FROM events_flat
  WHERE session_id IS NOT NULL
  GROUP BY user_pseudo_id, session_id
),

-- Rank sessions by time per user
ranked_sessions AS (
  SELECT
    user_pseudo_id,
    session_id,
    session_start_time,
    RANK() OVER (PARTITION BY user_pseudo_id ORDER BY session_start_time ASC) AS session_rank
  FROM session_start_times
),

user_firsttwo_sessions AS(
SELECT *
FROM ranked_sessions
WHERE session_rank <=4 -- Get only 4 session or less
ORDER BY user_pseudo_id ,session_rank ASC
),
user_metadata AS (
  SELECT
    user_pseudo_id,
    device.category        AS device_category,
    geo.country            AS country
  FROM (
    SELECT *,
      RANK() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC) AS session_rank
    FROM `gifted-country-461816-i9.ga4_sample_data.events_*`
    WHERE event_name = 'session_start'
  )
  WHERE session_rank = 1
)

SELECT
m.user_pseudo_id,
--Behavioural data
COUNTIF(event_name = 'page_view') AS page_view_count,
COUNTIF(event_name = 'view_item') AS view_item_count,
COUNTIF(event_name = 'scroll') AS scroll_count,
COUNTIF(event_name = 'user_engagement') AS user_engagement_count,
COUNTIF(event_name = 'add_to_cart') AS add_to_cart_count,
COUNTIF(event_name = 'view_promotion') AS view_promotion_count,
SUM(CAST((SELECT value.int_value
            FROM UNNEST(event_params)
            WHERE key = 'engagement_time_msec') AS INT64)) AS total_engagement_time_msec,
--demographic
  um.device_category AS device_category,
  um.country AS country,

--label
CASE WHEN (COUNTIF (event_name = 'purchase') >= 1 ) THEN 1 ELSE 0 END AS label
FROM `gifted-country-461816-i9.ga4_sample_data.events_*` m
JOIN user_firsttwo_sessions f
ON (SELECT value.int_value FROM UNNEST(event_params) where key = 'ga_session_id')
  = f.session_id
LEFT JOIN user_metadata um
ON  m.user_pseudo_id = um.user_pseudo_id
GROUP BY m.user_pseudo_id,um.device_category, um.country

  )
)

ORDER BY predicted_label DESC , purchase_probability DESC


In [ ]:
display(df)

#Recommendations & Next Steps

### 🎯 Model Refinement
- **Tune the classification threshold** (not just 0.5) to better balance precision and recall depending on your business objective (e.g. acquisition vs retention).
- **Reweight classes** to account for class imbalance (e.g. <2% of users made a purchase), improving the model’s sensitivity to rare events.

---

### 🧠 Feature Refinement
- **Normalise or log-transform** `total_engagement_time_msec` to reduce skew and improve model fit.
- **Create interaction features** such as `view_item_count * engagement_time` to capture user intensity.
- **Bin features** like `add_to_cart_count` into tiers to model behavioural stages (e.g. low, medium, high intent).
- **Integrate CRM tags or loyalty programme signals** to enrich understanding of customer type.

---

### ➕ Add More Contextual Features
- **Include digital marketing attributes** such as `traffic_source.medium`, `traffic_source.source`, and `utm_campaign` for acquisition insights. *(Note: not available in the current sample dataset.)*
- **Add session-level temporal features** such as `day_of_week`, `hour_of_day`, or `session_duration`.

---

### 🔄 Segment-Specific Modelling
- Train separate models by key segments such as:
  - `device.category` (e.g. mobile vs desktop)
  - `geo.country` or `geo.region`
- This captures behavioural nuances that may differ significantly across user cohorts.

---

### 📈 Business Applications
Leverage model predictions to drive real-world value:
- **Trigger remarketing campaigns** or personalised email sequences.
- **Tailor homepage/product suggestions** based on predicted conversion likelihood.
- **Identify high-potential visitors** for loyalty or incentive programmes.


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()